In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Embedding, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import random

# The text data you provided
text = """Buffalo Bill’s
defunct
who used to
ride a watersmooth-silver
stallion
and break one two three four five pigeons just like that
Jesus

he was a handsome man
and what i want to know is
how do you like your blueeyed boy
Mister Death

Had I the heaven’s embroidered cloths,
Enwrought with golden and silver light,
The blue and the dim and the dark cloths
Of night and light and the half-light,
I would spread the cloths under your feet:
But I, being poor, have only my dreams;
I have spread my dreams under your feet;
Tread softly because you tread on my dreams.

He clasps the crag with crooked hands;
Close to the sun in lonely lands,
Ring’d with the azure world, he stands.

The wrinkled sea beneath him crawls;
He watches from his mountain walls,
And like a thunderbolt he falls.

Some say the world will end in fire,
Some say in ice.
From what I’ve tasted of desire
I hold with those who favor fire.
But if it had to perish twice,
I think I know enough of hate
To say that for destruction ice
Is also great
And would suffice.

Two roads diverged in a yellow wood,
And sorry I could not travel both
And be one traveler, long I stood
And looked down one as far as I could
To where it bent in the undergrowth;

Then took the other, as just as fair,
And having perhaps the better claim,
Because it was grassy and wanted wear;
Though as for that the passing there
Had worn them really about the same,

And both that morning equally lay
In leaves no step had trodden black.
Oh, I kept the first for another day!
Yet knowing how way leads on to way,
I doubted if I should ever come back.

I shall be telling this with a sigh
Somewhere ages and ages hence:
Two roads diverged in a wood, and I—
I took the one less traveled by,
And that has made all the difference."""

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

# Create input sequences and labels
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences to have the same length
max_sequence_length = max([len(x) for x in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

# Create predictors and labels
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = to_categorical(y, num_classes=total_words)

# Build the GRU model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_length - 1))
model.add(GRU(150))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))

# Train the model
model.fit(X, y, epochs=100, verbose=1)

# Generate text
seed_text = "Some say the world will end in fire,"
next_words = 50

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
    predicted_probs = model.predict(token_list, verbose=0)
    predicted = np.argmax(predicted_probs, axis=-1)  # Select the word with the highest probability
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted[0]:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)


Epoch 1/100
9/9 [==============================] - 3s 18ms/step - loss: 5.2760
Epoch 2/100
9/9 [==============================] - 0s 18ms/step - loss: 5.2364
Epoch 3/100
9/9 [==============================] - 0s 18ms/step - loss: 5.1204
Epoch 4/100
9/9 [==============================] - 0s 20ms/step - loss: 4.9793
Epoch 5/100
9/9 [==============================] - 0s 19ms/step - loss: 4.9074
Epoch 6/100
9/9 [==============================] - 0s 18ms/step - loss: 4.8665
Epoch 7/100
9/9 [==============================] - 0s 18ms/step - loss: 4.8218
Epoch 8/100
9/9 [==============================] - 0s 20ms/step - loss: 4.7697
Epoch 9/100
9/9 [==============================] - 0s 19ms/step - loss: 4.6994
Epoch 10/100
9/9 [==============================] - 0s 19ms/step - loss: 4.5906
Epoch 11/100
9/9 [==============================] - 0s 19ms/step - loss: 4.4524
Epoch 12/100
9/9 [==============================] - 0s 18ms/step - loss: 4.3124
Epoch 13/100
9/9 [==============================]